In [1]:
# Install TensorFlow
!pip install tensorflow==2.0.0-beta1
!pip install matplotlib
!pip install sklearn


     |████████████████████████████████| 87.9MB 1.3MB/s 
     |████████████████████████████████| 501kB 37.4MB/s 
     |████████████████████████████████| 3.1MB 28.6MB/s 
  Found existing installation: tensorflow 1.14.0
    Uninstalling tensorflow-1.14.0:
      Successfully uninstalled tensorflow-1.14.0


In [0]:
## Code
import numpy as np
from sklearn import preprocessing 
import tensorflow as tf
import pandas as pd

## Collect the data

In [6]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
# data = pd.read_csv('/content/drive/My Drive/machine_learning_data/Audiobooks_data.csv')
raw_csv_data = np.loadtxt('/content/drive/My Drive/machine_learning_data/Audiobooks_data.csv', delimiter=',')

## data classes here are for balancing 2 categories

unscaled_inputs_all = raw_csv_data[:,1:-1]


unscaled_inputs_all = raw_csv_data[:,1:-1]

# The targets are in the last column. That's how datasets are conventionally organized.
targets_all = raw_csv_data[:,-1]


# original source: https://www.dropbox.com/sh/fdqk8i4crvb9ox4/AAB7oVvIDQ7-yOGXTJimyyaJa?dl=0&preview=Audiobooks_data.csv

In [8]:
unscaled_inputs_all[:10]

array([[2.1600e+03, 2.1600e+03, 1.0130e+01, 1.0130e+01, 0.0000e+00,
        8.9100e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00],
       [1.4040e+03, 2.8080e+03, 6.6600e+00, 1.3330e+01, 1.0000e+00,
        6.5000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 1.8200e+02],
       [3.2400e+02, 3.2400e+02, 1.0130e+01, 1.0130e+01, 1.0000e+00,
        9.0000e+00, 0.0000e+00, 0.0000e+00, 1.0000e+00, 3.3400e+02],
       [1.6200e+03, 1.6200e+03, 1.5310e+01, 1.5310e+01, 0.0000e+00,
        9.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 1.8300e+02],
       [4.3200e+02, 1.2960e+03, 7.1100e+00, 2.1330e+01, 1.0000e+00,
        9.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00],
       [2.1600e+03, 2.1600e+03, 1.0130e+01, 1.0130e+01, 1.0000e+00,
        9.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 5.0000e+00],
       [1.4040e+03, 2.8080e+03, 6.6700e+00, 1.3330e+01, 0.0000e+00,
        7.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 9.0000e+01],
       [1.2312e+03, 6.1560e+03, 7.1500e+0

## Balance Dataset

In [0]:
num_one_targets = int(np.sum(targets_all))

In [0]:
zero_targets_counter = 0

In [0]:
indices_to_remove = []

In [0]:
# Count the number of targets that are 0. 
# Once there are as many 0s as 1s, mark entries where the target is 0.
for i in range(targets_all.shape[0]):
    if targets_all[i] == 0:
        zero_targets_counter += 1
        if zero_targets_counter > num_one_targets:
            indices_to_remove.append(i)
# Create two new variables, one that will contain the inputs, and one that will contain the targets.
# We delete all indices that we marked "to remove" in the loop above.
unscaled_inputs_equal_priors = np.delete(unscaled_inputs_all, indices_to_remove, axis=0)
targets_equal_priors = np.delete(targets_all, indices_to_remove, axis=0)

## Scale Inputs

In [0]:
scaled_inputs = preprocessing.scale(unscaled_inputs_equal_priors)

## Shuffle things

In [0]:
# When the data was collected it was actually arranged by date
# Shuffle the indices of the data, so the data is not arranged in any way when we feed it.
# Since we will be batching, we want the data to be as randomly spread out as possible
shuffled_indices = np.arange(scaled_inputs.shape[0])
np.random.shuffle(shuffled_indices)

# Use the shuffled indices to shuffle the inputs and targets.
shuffled_inputs = scaled_inputs[shuffled_indices]
shuffled_targets = targets_equal_priors[shuffled_indices]

Split Data

In [21]:
samples_count = shuffled_inputs.shape[0]
train_samples_count = int(0.8*samples_count)
validation_samples_count = int(0.1*samples_count)
test_samples_count = samples_count - train_samples_count - validation_samples_count

train_inputs = shuffled_inputs[:train_samples_count]
train_targets = shuffled_targets[:train_samples_count]

validation_inputs = shuffled_inputs[train_samples_count:train_samples_count + validation_samples_count]
validation_targets = shuffled_targets[train_samples_count:train_samples_count + validation_samples_count]

test_inputs = shuffled_inputs[train_samples_count + validation_samples_count:]
test_targets = shuffled_targets[train_samples_count + validation_samples_count:]

## We want priors to be close to 50%

print(np.sum(train_targets), train_samples_count, np.sum(train_targets) / train_samples_count)
print(np.sum(validation_targets), validation_samples_count, np.sum(validation_targets) / validation_samples_count)
print(np.sum(test_targets), test_samples_count, np.sum(test_targets) / test_samples_count)


1799.0 3579 0.5026543727298128
217.0 447 0.4854586129753915
221.0 448 0.49330357142857145


In [0]:
np.savez('Audiobooks_data_train', inputs=train_inputs, targets=train_targets)
np.savez('Audiobooks_data_validation', inputs=validation_inputs, targets=validation_targets)
np.savez('Audiobooks_data_test', inputs=test_inputs, targets=test_targets)

